# 在Dynex平台上使用Python进行计算 - ACC

Dynex是基于DynexSolve芯片算法的全球首个神经形态超级计算区块链，采用了一种解决实际问题的“有用工作证明”（PoUW）方法。这个示例演示了如何使用Dynex SDK来使用Python在Dynex平台上进行计算

In [1]:
import dynex
import dimod

## 构建一个二次二元模型

二次二元模型（BQM）示例:

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mi>E</mi>
  <mo stretchy="false">(</mo>
  <mrow data-mjx-texclass="ORD">
    <mi mathvariant="bold">v</mi>
  </mrow>
  <mo mathvariant="bold" stretchy="false">)</mo>
  <mo mathvariant="bold">=</mo>
  <munder>
    <mo data-mjx-texclass="OP">&#x2211;</mo>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
    </mrow>
  </munder>
  <msub>
    <mi mathvariant="bold">a</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <mo mathvariant="bold">+</mo>
  <munder>
    <mo data-mjx-texclass="OP">&#x2211;</mo>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
      <mo mathvariant="bold">&lt;</mo>
      <mi mathvariant="bold">j</mi>
    </mrow>
  </munder>
  <msub>
    <mi mathvariant="bold">b</mi>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
      <mo mathvariant="bold">,</mo>
      <mi mathvariant="bold">j</mi>
    </mrow>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">j</mi>
  </msub>
  <mo mathvariant="bold">+</mo>
  <mi mathvariant="bold">c</mi>
  <mstyle scriptlevel="0">
    <mspace width="2em"></mspace>
  </mstyle>
  <mstyle scriptlevel="0">
    <mspace width="2em"></mspace>
  </mstyle>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <mo>&#x2208;</mo>
  <mo fence="false" stretchy="false">{</mo>
  <mo mathvariant="bold">&#x2212;</mo>
  <mn mathvariant="bold">1</mn>
  <mo mathvariant="bold">,</mo>
  <mo mathvariant="bold">+</mo>
  <mn mathvariant="bold">1</mn>
  <mo fence="false" stretchy="false">}</mo>
  <mtext mathvariant="bold">&#xA0;or&#xA0;</mtext>
  <mo fence="false" stretchy="false">{</mo>
  <mn mathvariant="bold">0</mn>
  <mo mathvariant="bold">,</mo>
  <mn mathvariant="bold">1</mn>
  <mo fence="false" stretchy="false">}</mo>
</math>

其中a、b、c为实数值.

我们需要定义一个代表问题的二次模型（QM）。构建一个二次二元模型（BQM）的最简单方法是使用dimod的符号变量。对于BQM中的每个数学变量，我们使用dimod定义一个符号二元变量。一旦QM被定义，它被存储为一个BinaryQuadraticModel对象。这个对象存储了数学表达式的线性和二次系数，任何常数项或偏移量，以及用于构建模型的变量类型。在这种情况下，我们构建的bqmmodel对象会显示以下内容：

以下示例用于减少一个QUBO问题的Ising模型，用于举例说明:

### 反交叉问题

生成一个带有单个圈子的反交叉问题。令N = num_variables // 2。这个函数返回一个二次二元模型，其中一半的变量，[0, N)，形成一个铁磁圈子，每个变量v也与剩余一半变量，[N, 2*N)中的变量v+N铁磁相互作用。

除了变量1以外，圈子中的所有变量都具有+1的线性偏置，附加到圈子的所有变量都具有-1的线性偏置。

In [2]:
bqmodel = dimod.generators.anti_crossing_clique(6);
print(bqmodel)


BinaryQuadraticModel({0: 1.0, 1: 0.0, 2: 1.0, 3: -1.0, 4: -1.0, 5: -1.0}, {(1, 0): -1.0, (2, 0): -1.0, (2, 1): -1.0, (3, 0): -1.0, (4, 1): -1.0, (5, 2): -1.0}, 0.0, 'SPIN')


根据我们的BQM定义Dynex BQM模型

In [3]:
model = dynex.BQM(bqmodel);

[DYNEX] MODEL CONVERTED TO QUBO
[DYNEX] PRECISION SET TO 0.0001
[DYNEX] QUBO: Constant offset of the binary quadratic model: -5.0


## 与Dynex采样器互动

为了找到QM的最小能量状态（即为我们的QM提供最小能量值的变量值分配），Dynex SDK提供了采样器和求解器。求解器是运行问题的资源。采样器是运行问题多次以获取样本集合的过程，每个样本都是我们问题的一个可能解决方案。为了方便起见，通常我们会将Dynex的采样器整体称为一个术语，以包括求解器。

In [4]:
sampler = dynex.DynexSampler(model, mainnet=False);

[DYNEX] SAMPLER INITIALISED


一旦我们在程序中建立了我们的采样器，我们就可以为我们的QM调用它。每种类型的QM模型都有自己的与采样器交互的方法，无论是QUBO、BinaryQuadraticModel还是其他任何QM。我们使用Dynex的采样函数之一来调用采样器来对我们的QM进行采样，具体取决于我们使用的QM类型。例如，下面的代码片段演示了如何使用Dynex平台对名为bqm的BinaryQuadraticModel对象进行采样。

In [5]:
sampleset = sampler.sample(num_reads=32, annealing_time = 100);

[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬───────────┬───────────────────────────┬─────────┬─────┬─────────┬───────┬─────┬──────────┬─────╮
│ DYNEXJOB   │ ELAPSED   │ WORKERS                   │ CHIPS   │ ✔   │ STEPS   │ LOC   │ ✔   │ ENERGY   │ ✔   │
├────────────┼───────────┼───────────────────────────┼─────────┼─────┼─────────┼───────┼─────┼──────────┼─────┤
│            │           │ *** WAITING FOR READS *** │         │     │         │       │     │          │     │
╰────────────┴───────────┴───────────────────────────┴─────────┴─────┴─────────┴───────┴─────┴──────────┴─────╯

[DYNEX] FINISHED READ AFTER 0.00 SECONDS
[DYNEX] SAMPLESET LOADED


在我们对QM进行采样后，采样器会返回一个SampleSet对象。该对象包含所有返回的样本及其相应的能量值、芯片数量、积分步数等信息。附加信息的内容取决于使用的采样器。随着用户对Dynex SDK和各种可用的采样器变得更加熟悉，花一些时间来探索SampleSet对象中提供的大量信息通常是有用的。我们可以访问SampleSet的一些关键属性和方法，包括以下内容：

In [6]:
print(sampleset.first) # 输出能量最低的样本

Sample(sample={0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1}, energy=-7.0, num_occurrences=1)


The sample shows the corresponding energy values for our QM and additional information like total energy, number of chips or number of integration steps.

In [8]:
print(sampleset.first.sample) # 0到5的能量水平:

{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1}
